In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

import torch

from autogen import ConversableAgent, GroupChat
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

from Utl import process_locs_buildings, fetch_acs_county_data, write_to_csv, read_from_csv
import copy
import json

In [11]:
torch.cuda.is_available()

True

In [ ]:
from dataclasses import dataclass


@dataclass
class ExecutorGroupchat(GroupChat):
    def select_speaker(
        self, last_speaker: ConversableAgent, selector: ConversableAgent
    ):
        """Select the next speaker."""

        try:
            message = self.messages[-1]
            if "function_call" in message:
                return self.admin
        except Exception as e:
            print(e)
            pass

        selector.update_system_message(self.select_speaker_msg())
        final, name = selector.generate_oai_reply(
            self.messages
            + [
                {
                    "role": "system",
                    "content": f"Read the above conversation. Then select the next role from {self.agent_names} to play. Only return the role.",
                }
            ]
        )
        if not final:
            # i = self._random.randint(0, len(self._agent_names) - 1)  # randomly pick an id
            return self.next_agent(last_speaker)
        try:
            return self.agent_by_name(name)
        except ValueError:
            return self.next_agent(last_speaker)

# Individual Level

### Agents Setups

In [70]:
llm_config = {
    "config_list": [
        {
            "model": "llama3.1:8b",
            "base_url": "http://localhost:11434/v1",
            "api_key": "ollama",
        }
    ]
}

# 1) Population Manager Agent (PMA)
population_manager = ConversableAgent(
    name="PopulationManager",
    system_message=(
        "You are the Population Manager."
        "Your task: generate exactly 10 personal profiles with the following four attributes:"
        "  • age_class (from the given age distribution),"
        "  • gender (from the given gender distribution),"
        "  • education_level (from the given education distribution),"
        "  • income_class (from the given income distribution)."
        "Ensure the overall sample of 10 matches the input distributions as closely as possible,"
        "and that each individual combination is plausible (e.g. age vs. income/education)."
        "Here are the target distributions:"
        "Age: {age_dist}"
        "Education: {edu_dist}\n"
        "Income: {income_dist}\n"
        "Gender: {gender_dist}\n"
        "Please do not write code but just generate the profiles."
        "Once the task is complete, return 'TERMINATE'."
    ),
    llm_config=llm_config,
)

# 2) Activity Manager Agent (AMA)
activity_manager = ConversableAgent(
    name="ActivityManager",
    system_message=(
        """You are the Activity Manager.

Input 10 personal profiles with the following attributes:
1. age_class: '20-24', gender: 'Female', education_level: 'Bachelor', income_class: '35-50k'
2. age_class: '45-49', gender: 'Male', education_level: 'High school', income_class: '<10k'
3. age_class: '30-34', gender: 'Female', education_level: 'Graduate', income_class: '75-100k'
4. age_class: '25-29', gender: 'Male', education_level: 'Some college', income_class: '150-200k'
5. age_class: '10-14', gender: 'Female', education_level: 'High school', income_class: '<10k'
6. age_class: '40-44', gender: 'Male', education_level: 'Bachelor', income_class: '50-75k'
7. age_class: '55-59', gender: 'Female', education_level: 'Some college', income_class: '25-35k'
8. age_class: '15-19', gender: 'Male', education_level: 'Graduate', income_class: '100-150k'
9. age_class: '65-69', gender: 'Female', education_level: 'High school', income_class: '<10k'
10. age_class: '35-39', gender: 'Male', education_level: 'Bachelor', income_class: '75-100k'

Input list of buildings:
[B1,B2,B3,B4,B5]

Task:
For each agent, generate a 24-hour schedule (00:00-24:00) as a contiguous list of activities. Each activity must include:
  • type (e.g., “sleep”, “commute”, “work”, “meal”, “errand”, “leisure”, “exercise”)
  • start_time (HH:MM)
  • end_time (HH:MM)
  • duration (minutes)
  • building_id (one of the provided buildings)

Rules:
1. First activity starts at “00:00”, last ends at “24:00”, with no gaps or overlaps.
2. Sleep ≥6 hours, scheduled between 20:00-10:00.
3. Meals: breakfast 06:00-09:00, lunch 11:00-14:00, dinner 17:00-20:00.
4. Commute must begin or end at an agent's “home” building (choose any building_id to represent home).
5. Distribute work/school block, errands, and leisure realistically by age_class and income_class.

Output:
Present each diary as:
  
  Agent <id>:
    00:00-06:30  Sleep at Home (B12)
    06:30-07:00  Commute to Office (B3)
    07:00-12:00  Work at Office (B3)
    12:00-12:45  Lunch at Café (B9)
    … 
    23:00-24:00  Wind down at Home (B12)
  
Output:
Return exactly 10 diaries—one per agent—in this format. 
Once the task is complete, return 'TERMINATE'."""
    ),
    llm_config=llm_config,
)

# 3) Interaction Manager Agent (IMA)
interaction_manager = ConversableAgent(
    name="InteractionManager",
    system_message=(
        """You are the Interaction Manager.

Input:  
A block of 10 agent diaries in this format:


Task:
1. Parse each agent's schedule.
2. Find any pair of agents who are at the **same building_id** for **at least 30 consecutive minutes**.
3. For each such overlap, decide “meet” or “no meet” (they meet if they share age bracket or plausible social affinity).
4. If “meet,” insert a new 20-minute activity block labelled:
     `Social Meetup at [building_id]`  
   positioned at the midpoint of that overlap.
5. Preserve the diary formatting.  

Output:
Return the 10 updated diaries in the same plain-text style, with any “Social Meetup” lines injected."""
    ),
    llm_config=llm_config,
)

# 4) Mobility Manager Agent (MMA)
mobility_manager = ConversableAgent(
    name="MobilityManager",
    system_message=(
        "You are the Mobility Manager. "
        "Given updated agent schedules (including interactions), "
        "choose transport modes, compute routes, and output travel itineraries."
    ),
    llm_config=llm_config,
)

# 5) Worker Agent (WA)
worker = ConversableAgent(
    name="WorkerAgent",
    system_message="You are the Worker. Execute exactly the prompt you receive and return the result verbatim.",
    llm_config=llm_config,
)

# 6) Excute Agent (EA)
user_proxy = ConversableAgent(
    name="User",
    system_message="Help the agent generate data, and terminate when the activities have been generated",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [ ]:
population_manager.register_for_llm(name="fetch_acs_county_data", description="fetch_acs_county_data")(fetch_acs_county_data)
activity_manager.register_for_llm(name="process_locs_buildings", description="process_locs_buildings")(process_locs_buildings)

user_proxy.register_for_execution(name="fetch_acs_county_data")(fetch_acs_county_data)
user_proxy.register_for_execution(name="process_locs_buildings")(process_locs_buildings)

### Population

In [24]:
age_dist = {
    "Under 5": 18358199, "5-9": 19770314, "10-14": 21220214,
    "15-19": 21785658, "20-24": 22446040, "25-29": 22007628,
    "30-34": 23225636, "35-39": 22335859, "40-44": 21711785,
    "45-49": 19640925, "50-54": 20835521, "55-59": 20553615,
    "60-64": 21573853, "65-69": 18559497, "70-74": 15338575,
    "75-79": 11005461, "80-84": 6758839, "85+": 6159943
}
edu_dist = {
    "High school": 71249465,
    "Some college": 75938457,
    "Bachelor": 54661306,
    "Graduate": 33248210
}
income_dist = {
    "<10k": 60940350, "10-15k": 2103145, "15-25k": 7111313,
    "25-35k": 13282796, "35-50k": 22799035, "50-75k": 20007019,
    "75-100k": 16244851, "100-150k": 16710010, "150-200k": 8355005,
    "200k+": 2785001, "NotWorking": 153314315
}
gender_dist = {"Male":165228214, "Female":168059348}

dist_prompt = population_manager.system_message.format(
    age_dist=age_dist,
    edu_dist=edu_dist,
    income_dist=income_dist,
    gender_dist=gender_dist
)


In [28]:
response = user_proxy.initiate_chat(recipient=population_manager,
    message=dist_prompt
)

User (to PopulationManager):

You are the Population Manager.
Your task: generate exactly 10 personal profiles with the following four attributes:
  • age_class (from the given age distribution),
  • gender (from the given gender distribution),
  • education_level (from the given education distribution),
  • income_class (from the given income distribution).
Ensure the overall sample of 10 matches the input distributions as closely as possible,
and that each individual combination is plausible (e.g. age vs. income/education).

Here are the target distributions:
Age: {'Under 5': 18358199, '5-9': 19770314, '10-14': 21220214, '15-19': 21785658, '20-24': 22446040, '25-29': 22007628, '30-34': 23225636, '35-39': 22335859, '40-44': 21711785, '45-49': 19640925, '50-54': 20835521, '55-59': 20553615, '60-64': 21573853, '65-69': 18559497, '70-74': 15338575, '75-79': 11005461, '80-84': 6758839, '85+': 6159943}
Education: {'High school': 71249465, 'Some college': 75938457, 'Bachelor': 54661306, 'Gr

In [32]:
response

ChatResult(chat_id=None, chat_history=[{'content': "You are the Population Manager.\nYour task: generate exactly 10 personal profiles with the following four attributes:\n  • age_class (from the given age distribution),\n  • gender (from the given gender distribution),\n  • education_level (from the given education distribution),\n  • income_class (from the given income distribution).\nEnsure the overall sample of 10 matches the input distributions as closely as possible,\nand that each individual combination is plausible (e.g. age vs. income/education).\n\nHere are the target distributions:\nAge: {'Under 5': 18358199, '5-9': 19770314, '10-14': 21220214, '15-19': 21785658, '20-24': 22446040, '25-29': 22007628, '30-34': 23225636, '35-39': 22335859, '40-44': 21711785, '45-49': 19640925, '50-54': 20835521, '55-59': 20553615, '60-64': 21573853, '65-69': 18559497, '70-74': 15338575, '75-79': 11005461, '80-84': 6758839, '85+': 6159943}\nEducation: {'High school': 71249465, 'Some college': 75

1. age_class: '20-24', gender: 'Female', education_level: 'Bachelor', income_class: '35-50k'
2. age_class: '45-49', gender: 'Male', education_level: 'High school', income_class: '<10k'
3. age_class: '30-34', gender: 'Female', education_level: 'Graduate', income_class: '75-100k'
4. age_class: '25-29', gender: 'Male', education_level: 'Some college', income_class: '150-200k'
5. age_class: '10-14', gender: 'Female', education_level: 'High school', income_class: '<10k'
6. age_class: '40-44', gender: 'Male', education_level: 'Bachelor', income_class: '50-75k'
7. age_class: '55-59', gender: 'Female', education_level: 'Some college', income_class: '25-35k'
8. age_class: '15-19', gender: 'Male', education_level: 'Graduate', income_class: '100-150k'
9. age_class: '65-69', gender: 'Female', education_level: 'High school', income_class: '<10k'
10. age_class: '35-39', gender: 'Male', education_level: 'Bachelor', income_class: '75-100k'

### Activity Generation

In [36]:
act_dist_prompt = activity_manager.system_message.format(

)

In [69]:
response = user_proxy.initiate_chat(recipient=activity_manager,
    message=act_dist_prompt
)

User (to ActivityManager):

You are the Activity Manager.

Input 10 personal profiles with the following attributes:
1. age_class: '20-24', gender: 'Female', education_level: 'Bachelor', income_class: '35-50k'
2. age_class: '45-49', gender: 'Male', education_level: 'High school', income_class: '<10k'
3. age_class: '30-34', gender: 'Female', education_level: 'Graduate', income_class: '75-100k'
4. age_class: '25-29', gender: 'Male', education_level: 'Some college', income_class: '150-200k'
5. age_class: '10-14', gender: 'Female', education_level: 'High school', income_class: '<10k'
6. age_class: '40-44', gender: 'Male', education_level: 'Bachelor', income_class: '50-75k'
7. age_class: '55-59', gender: 'Female', education_level: 'Some college', income_class: '25-35k'
8. age_class: '15-19', gender: 'Male', education_level: 'Graduate', income_class: '100-150k'
9. age_class: '65-69', gender: 'Female', education_level: 'High school', income_class: '<10k'
10. age_class: '35-39', gender: 'Male'

Agent 1:
 Age: 20-24
Gender: Female
Education Level: Bachelor
Income Class: 35-50k
Home Building ID: B3 (randomly assigned)

Diary:
00:00–07:30 Sleep at Home (B3)
07:30–08:00 Commute to Office (B3)
08:00–14:00 Work at Office (B3)
14:00–15:00 Leisure at Coffee Shop (B5)
15:00-17:00  Errand - grocery shopping, at Building B2
17:00–18:30 Dinner at Restaurant
18:30–20:00 Walk/Bike ride for leisure at Home 
20:00–02:30 Sleep at Home (B3)

Agent 2:
 Age: 45-49
Gender: Male
Education Level: High school
Income Class: <10k
Home Building ID: B5 (randomly assigned)

Diary:
00:00–12:00 Work at Factory (B1)
12:00–18:00 errands and meals, B2, and leisure activities  walking/hiking 
18:00- 20:30 walk/ bike exercise at B4
20:30 –08:30 sleep

Agent 3:
 Age: 30-34
Gender: Female
Education Level: Graduate
Income Class: 75-100k
Home Building ID: B2 (randomly assigned)

Diary:
00:00–05:30 Sleep at Home (B2)
05:30–06:30 Commute to Office (B3)
06:30–12:00 Work at Office (B3)
12:00- 14:00 Break/ Lunch, B9 
14:00 –16:00 Online Course / Study for certification at self
16:00 –18:00 walk/ bike leisure time at Home B2 
18:00 –20:30 Errands/ Grocery shopping and dinner at a restaurant.
22:00- 08:00 sleep

Agent 4:
 Age: 25-29
Gender: Male
Education Level: Some college
Income Class: 150-200k
Home Building ID: B1 (randomly assigned)

Diary:
00:00 –06:30 Sleep at Home (B1)
06:30–07:30 Commute to Office (B3)
07:30-19:30 Work, 
18:30 –20:00 Free time,
20:00- 22:30 leisure and fun activities,  
23:00 - 08:00 sleep

Agent 5:
 Age: 10-14
Gender: Female
Education Level: High school
Income Class: <10k
Home Building ID: B1 (randomly assigned)

Diary: 
00:00 –05:30 Sleep at Home (B11)
05:30–06:30 Commute to School (B2)
06:30-17:30 Attend Classes, including recess and lunch 
17:30 –18:30 leisure/ study / hobby time
 18:30 - 20:30 Walks with family,
21:00 - 07 :00 sleep 

Agent 6:
 Age: 40-44
Gender: Male
Education Level: Bachelor
Income Class:  50-75k 
Home Building ID: B5 (randomly assigned)

Diary: (some changes were made to account for older adult needs)
00:00 -10:30 sleep at Home (B12), due to late work hours. 
12:00 –13:00 Lunch/ break
13:00-18:00 Office Hours
 18:00 –20:00 Free time, with walk/ bike leisure
20:00- 21:45 dinner and errands 
10:00(30) -06 :00 sleep 

Agent 7:
 Age: 55-59
Gender: Female
Education Level: Some college
Income Class: 25-35k 
Home Building ID: B4 (randomly assigned)

Diary: (similar reasoning for Agent 6; a mix of flexibility with time allocated to leisure, errands, mealtimes)
00:00 -08:30 sleep at home (B11), due to health and energy constraints 
17:15- 19:00 Leisure/ hobbies
19:00-22 :00 socializing / evening walk/Biking 
22:00 –06:40 (30 min) sleep 

Agent 8:
 Age: 15-19
Gender: Male
Education Level: Graduate
Income Class: 100-150k 
Home Building ID: B4 (randomly assigned)

Diary:
00:00–08:30 Sleep at Home (B12), considering school schedule, and late night work/study 
11:00 –14:00 Attend classes - study/homework 
16:30-   Walk/Bike exercise 
21:00 dinner with friends,
23:59  sleep

Agent 9:
 Age: 65-69
Gender: Female
Education Level: High school
Income Class: <10k  
Home Building ID: B1 (randomly assigned)

Diary: Considering older adult needs, and schedule adjustments in terms of energy levels 
12:30–14 :00 Attend to chores / errands,
22:30 – 06:15   Sleep 

Agent 10:
 Age: 35-39
Gender: Male
Education Level: Bachelor
Income Class: 75-100k  
Home Building ID: B2 (randomly assigned)  

Diary:
00:00–05 :00 /14:00 sleep at home(considering work timing)
12:30 –14:15 Lunch 
14:00 -19:45   Work shift.
19:45- 20:30 leisure and hobby time
22:00/9–5  sleep 

### Interaction Update

# Aggregated level

In [76]:
aggregated_activity_manager = ConversableAgent(
    name="AggregatedActivityManager",
    system_message=(
        """You are the Aggregated Activity Manager.
      Input:
      1.  `clusters` of population clusters. Each cluster has:
        - cluster_id (string)
        - population_count (int)
        - demographic_summary (e.g. age & income distributions within cluster)

      2. `activity_types`, listing possible activities, e.g.
        ["sleep","commute","work","meal","errand","leisure","exercise"]

      Task:
      For each cluster, produce an aggregated activity profile over 24 hours:
        • Divide the day into hourly bins (0-1, 1-2, …, 23-24).
        • For each bin and each activity_type, estimate the fraction of that clusters population
          engaged in that activity during that hour.
        • Fractions for each cluster must sum to 1.0 across all activities*hours.

      Once the task is complete, return 'TERMINATE'.
      """
    ),
    llm_config=llm_config,
)

In [73]:
prompt = ("You are the Aggregated Activity Manager. Input: five population clusters described below and a list of possible activities. "
          "Here are five population clusters: "
          "Cluster C1: Total population 5000, Age distribution 20–29 30%, 30–39 40%, 40–49 20%, 50–59 10%, Income distribution < $10k 10%, $10–25k 15%, $25–50k 40%, $50–100k 25%, > $100k 10%. "
          "Cluster C2: Total population 8000, Age distribution 0–9 12%, 10–19 18%, 20–29 20%, 30–39 25%, 40–49 15%, 50+ 10%, Income distribution < $10k 20%, $10–25k 30%, $25–50k 25%, $50–100k 15%, > $100k 10%. "
          "Cluster C3: Total population 6500, Age distribution 30–39 35%, 40–49 30%, 50–59 20%, 60+ 15%, Income distribution < $10k 8%, $10–25k 22%, $25–50k 30%, $50–100k 25%, > $100k 15%. "
          "Cluster C4: Total population 4200, Age distribution 0–9 10%, 10–19 15%, 20–29 25%, 30–39 20%, 40–49 15%, 50+ 15%, Income distribution < $10k 25%, $10–25k 35%, $25–50k 25%, $50–100k 10%, > $100k 5%. "
          "Cluster C5: Total population 7300, Age distribution 18–24 18%, 25–34 32%, 35–44 28%, 45–54 15%, 55+ 7%, Income distribution < $10k 12%, $10–25k 18%, $25–50k 30%, $50–100k 25%, > $100k 15%. "
          "Activity types to consider: sleep, commute, work, meal, errand, leisure, exercise. "
          "Task: For each cluster produce an aggregated 24-hour activity profile by hour from 0 to 24. For each hour and each activity type estimate the fraction of that cluster’s population engaged in that activity. Ensure fractions sum to 1 across all activities per hour. "
          "Output: return plain-text lines in the format ClusterID | Hour | sleep: x.xx | commute: x.xx | work: x.xx | meal: x.xx | errand: x.xx | leisure: x.xx | exercise: x.xx")

In [63]:
prompt = ""

In [67]:
prompt

''

In [78]:
response = user_proxy.initiate_chat(recipient=aggregated_activity_manager,
    message=prompt
)

User (to AggregatedActivityManager):

You are the Aggregated Activity Manager. Input: five population clusters described below and a list of possible activities. Here are five population clusters: Cluster C1: Total population 5000, Age distribution 20–29 30%, 30–39 40%, 40–49 20%, 50–59 10%, Income distribution < $10k 10%, $10–25k 15%, $25–50k 40%, $50–100k 25%, > $100k 10%. Cluster C2: Total population 8000, Age distribution 0–9 12%, 10–19 18%, 20–29 20%, 30–39 25%, 40–49 15%, 50+ 10%, Income distribution < $10k 20%, $10–25k 30%, $25–50k 25%, $50–100k 15%, > $100k 10%. Cluster C3: Total population 6500, Age distribution 30–39 35%, 40–49 30%, 50–59 20%, 60+ 15%, Income distribution < $10k 8%, $10–25k 22%, $25–50k 30%, $50–100k 25%, > $100k 15%. Cluster C4: Total population 4200, Age distribution 0–9 10%, 10–19 15%, 20–29 25%, 30–39 20%, 40–49 15%, 50+ 15%, Income distribution < $10k 25%, $10–25k 35%, $25–50k 25%, $50–100k 10%, > $100k 5%. Cluster C5: Total population 7300, Age distribu

ClusterC1 | 0-1 | sleep: 0.8 | commute: 0 | work: 0 | meal: 0 | errand: 0 | leisure: 0 | exercise: 0
ClusterC1 | 1-2 | sleep: 0.7 | commute: 0 | work: 0 | meal: 0 | errand: 0 | leisure: 0 | exercise: 0
ClusterC1 | 2-3 | sleep: 0.7 | commute: 0 | work: 0 | meal: 0 | errand: 0 | leisure: 0.3 | exercise: 0
ClusterC1 | 3-4 | commute: 0.1 | work: 0.3 | meal: 0.05 | errand: 0.05 | leisure: 0.03 | exercise: 0.02 | sleep: 0.5
ClusterC1 | 4-5 | commute: 0.15 | work: 0.1 | meal: 0.02 | errand: 0.01 | leisure: 0.005 | exercise: 0.005 | sleep: 45
ClusterC1 | 5-6 | commute: 0.8 | work: 0.15 | meal: 0.05 | errand: 0.02 | leisure: 0.015 | exercise: 0.01 | sleep: 0
ClusterC1 | 6-7 | commute: 0.5 | work: 0.3 | meal: 0.1 | errand: 0.05 | leisure: 0.025 | exercise: 0.02 | sleep: 0
 ClusterC1 | 7-8 | work: 0.4 | meal: 0.2 | errand: 0.1 | commute: 0.3 | leisure: 0.05 | exercise: 0.01 | sleep: 0
ClusterC1 | 8-9 | work: 0.5 | meal: 0.3 | errand: 0.1 | commute: 0.1 | leisure: 0.025 | exercise: 0.015 | sleep: 0
ClusterC1 | 9-10 | work: 0.4 | meal: 0.35 | errand: 0.05 | commute: 0.15 | leisure: 0.05 | exercise: 0.005 | sleep: 0
ClusterC1 | 10-11 | work: 0.6 | meal: 0.2 | errand: 0.1 | commute: 0.1 | leisure: 0.025 | exercise: 0.035 | sleep: 0
ClusterC1 | 11-12 | work: 0.55 | meal: 0.25 | errand: 0.075 | commute: 0.15 | leisure: 0.04 | exercise: 0.02 | sleep: 0
ClusterC1 | 12-13 | work: 0.65 | meal: 0.2 | errand: 0.05 | commute: 0.125 | leisure: 0.03 | exercise: 0.025 | sleep: 0
ClusterC1 | 13-14 | work: 0.75 | meal: 0.15 | errand: 0.075 | commute: 0.125 | leisure: 0.023 | exercise: 0.01 | sleep: 0
 ClusterC1 | 14-15 | work: 0.8 | meal: 0.2 | errand: 0.05 | commute: 0.15 | leisure: 0.005 | exercise: 0.025 | sleep: 0
ClusterC1 | 15-16 | work: 0.85 | meal: 0.12 | errand: 0.03 | commute: 0.125 | leisure: 0.015 | exercise: 0.01 | sleep: 0
ClusterC1 | 16-17 | work: 0.9 | meal: 0.15 | errand: 0.05 | commute: 0.15 | leisure: 0.006 | exercise: 0.009 | sleep: 0
 ClusterC1 | 17-18 | work: 0.85 | meal: 0.2 | errand: 0.06 | commute: 0.165 | leisure: 0.01 | exercise: 0.005 | sleep: 0
ClusterC1 | 18-19 | work: 0.8 | meal: 0.25 | errand: 0.075 | commute: 0.145 | leisure: 0.012 | exercise: 0.015 | sleep: 0
 ClusterC1 | 19-20 | work: 0.75 | meal: 0.25 | errand: 0.1 | commute: 0.125 | leisure: 0.01 | exercise: 0.02 | sleep: 0
ClusterC1 | 20-21 | commute: 0.35 | work: 0.3 | meal: 0.2 | errand: 0.05 | leisure: 0.03 | exercise: 0.025 | sleep: 0.05
 ClusterC1 | 21-22 | commute: 0.55 | work: 0.25 | meal: 0.15 | errand: 0.03 | leisure: 0.02 | exercise: 0.045 | sleep: 0
 ClusterC1 | 22-23 | commute: 0.6 | work: 0.225 | meal: 0.175 | errand: 0.05 | leisure: 0.01 | exercise: 0.04 | sleep: 0 
ClusterC1 | 23-24 | commute: 0.7 | work: 0.2 | meal: 0.13 | errand: 0.025 | leisure: 0.015 | exercise: 0.03 | sleep: 0
ClusterC1 | 24 | commute: 0.5 | work: 0.05 | meal: 0.225 | errand: 0.075 | leisure: 0.02 | exercise: 0.0132 | sleep: 0.125 

ClusterC2 | 0-1 | sleep: 0.9 | commute: 0 | work: 0 | meal: 0 | errand: 0 | leisure: 0 | exercise: 0
 ClusterC2| 1-2 | sleep: 0.8 | commute: 0.05 | work: 0.1 | meal: 0.05 | errand: 0.005 | leisure: 0.025 | exercise: 0.015
ClusterC2 | 2-3 | sleep: 0.7 | commute: 0.175 | work: 0.275 | meal: 0.075 | errand: 0.03 | leisure: 0.04 | exercise: 0.0125 
ClusterC2 | 3-4 | commute: 0.55 | work: 0.25 | meal: 0.1 | errand: 0.05 | leisure: 0.02 | exercise: 0.016667 | sleep: 0.011111
ClusterC2 | 4-5 | commute: 0.85 | work: 0.125 | meal: 0.075 | errand: 0.04 | leisure: 0.01 | exercise: 0.008333 | sleep: 0
 ClusterC2 | 5-6 | commute: 0.75 | work: 0.15 | meal: 0.125 | errand: 0.065 | leisure: 0.025 | exercise: 0.009259 | sleep: 0.002083 
ClusterC2 | 6-7 | commute: 0.6 | work: 0.25 | meal: 0.12 | errand: 0.06 | leisure: 0.033333 | exercise: 0.011111 | sleep: 0.004167
 ClusterC2| 7-8 | work: 0.5 | meal: 0.27 | errand: 0.07 | commute: 0.15 | leisure: 0.04 | exercise: 0.0125 | sleep: 0
ClusterC2| 8-9 | work: 0.45 | meal: 0.25 | errand: 0.08 | commute: 0.175 | leisure: 0.03 | exercise: 0.0125 | sleep: 0
 ClusterC2| 9-10 | work: 0.55 | meal: 0.3 | errand: 0.06 | commute: 0.18 | leisure: 0.025 | exercise: 0.016667 | sleep: 0
ClusterC2 | 10-11 | commute: 0.4 | work: 0.35 | meal: 0.195 | errand: 0.065 | leisure: 0.02 | exercise: 0.03 | sleep: 0
 ClusterC2 | 11-12 | commute: 0.45 | work: 0.27 | meal: 0.25 | errand: 0.083333 | leisure: 0.021429 | exercise: 0.025 | sleep: 0 
ClusterC2| 12-13 | commute: 0.7 | work: 0.28 | meal: 0.225 | errand: 0.07 | leisure: 0.02 | exercise: 0.017857 | sleep: 0
ClusterC2 | 13-14 | commuter: 0.55 | work: 0.235 | erand: 0.075 | meal: 0.25 | leisure: 0.029167 | exrecise: 0.02 
 ClusterC2| 14-15 | commute: 0.6 | work: 0.3 | errand: 0.09 | mea: 0.19l | leisure: 0.05 | exercise: 0.025 | sleep: 0
ClusterC2 | 15-16 | commute: 0.5 | work: 0.35 | meal: 0.235 | errand: 0.035 | leisure: 0.015 | exercise: 0.02 | sleep: 0 
 ClusterC2| 16-17 | commute: 0.6 | work: 0.31 | mea;l: 0.195 | errad|: 0.055 | leisure: 0.04l | exercise: 0.02778l | sleep: 0
ClusterC2 | 17-18 | commute: 0.55 | work: 0.325 | meal: 0.225 | errand: 0.075 | leisure: 0.033333 | exercise: 0.02388 | sleep: 0
 ClusterC2| 18-19 | commute: 0.35 | work: 0.31 | meal: 0.25 | errand: 0.065 | leisulre: 0.055 | exercijsse : 0.035l7 | sleep: 0
ClusterC2| 19-20 | commute: 0.5 | work: 0.24 | meal: 0.275 | errand: 0.08 | leisure: 0.0304 | exercise: 0.0241 | sleep: 0
 ClusterC2 | 20-21 | commute: 0.55 | work: 0.28 | meal: 0.195 | errand: 0.0455 | leisure: 0.03881 | excercise: 0.0258| sleep: 0.
ClusterC2 | 21-22 | commute: 0.475 | work: 0.35 | meal : 0.185 | errand: 0.05s | leisure: 0.0236l | exerciuse: 0.02| sleep: 0
 ClusterC2| 22-23 | commute: 0.375 | work: 0.32 | meal : 0.22sl | erradn: 0.054 | leisurer: 0.05l | exercise: 0.022 | sleep: 0 
ClusterC2 | 23-24| commute: 0.52 | work  28 | meal  0.19| errand 0.06 | leisure 0.025 | exercose 0.0125| sleep: 0
 ClusterC2 | 24s'| commute : 0.4s | work : 02l2 | meal  0.26sl | errand  075a | leisure : 0.024l | exercise : 0.01417 :sleep: 0.



ClusterC3 | 0-1 | sleep: 0.85 | commute: 0 | work: 0 | meal: 0 | errand: 0 | leisure: 0 | exerciase: 0
 ClusterC3| 1-2 | sleep: 0.8 | commute: .06l | work: 0.12s | meal:0 .07 | errnd: 0.015 | leisure :0.015 | exercise :0.01 
ClusterC3 | 2-3 | sleep: 0.7 | commute:0 .17 | work: 0.27l| mdel: 0.12s | erradn:0.05 | leisur e: 0.03 | exercise: 0.0125 
ClusterC3 | 3-4 | commute:0 :45l | wokr.: 02sl | meal  0.15s | errad : .08  | leisure : .03 | exerciuse:0.04 | sleep  : 0.008
 ClusterC3| 4-5 | commute:1 s.75 | work: .215 | meal:0 .165 | errand  075s | leisur el 045l | exersic e:0.03s | sleop:.0129
ClusterC3 | 5-6 | commut.: 0.7 | wkrk :  0.25 | meal :  0.25| errnd: .08 | leisur e :.04s | exercise :0.02s | sleep:0.00476 
 ClusterC3 | 6-7 | work : .45 :meal:: 0..30 | erreand  : 085s | commute:.175 | leisu rer : .05s | exercise : .03 | sleeper: 0r:0133
ClusterC3 | 7-8 | work :  46l | meal 0.27l | errnd:  .09ls | comute:0.1r | leisure:.07  | exercise :0.:025s | sleep: 0062 
 ClusterC3| 8-9 | work .:48 | meal | erand  099 | commuter:: 0.1 | leiure: .04s | exerciise | sleeper : 012l
ClusterC3 | 9-10 | commute.:6 .52 | eradn:.09 | commut:.195 | work  :45 | meal|0..28l | leisre:0.035 | exercise: 0.015 | sleep:0.0116
 ClusterC2|10-| commute:0.6 | wkrk 44sl | meal 0.235 | errnd  :075s | leisure .07 | exerciise:.02 |sleep: 0124
ClusterC3 | 11-12 | commuter 56ls | work: 40l | meal 0.22| erand:0.06 | leisurse :0.05l | excercsie | sleep: .0128 
 ClusterC3 | 12-13 | commute:.58  | wokr:.34s|l meal ;..17sr | erradn :09.:05l |leiaure:0.ls: | exercise0.0225 :sleep:0132
ClusterC3 | 14--15 | commute: 5.s7l | work 30sl | mea: .23| errnd: 06 | leisure:.03l | exrerice.024 | sleep:0.008 
 ClusterC3 | 15-16 s commute 54.l | wokr:32sl | meal:22s lr | erradn 07 l | leisure :031| exercise: .0225l | slepepr| 0094
ClusterC3 | 16-17 | commuter:.55 | work :30. | mall:.23| errand:0.06 | leisue  .0367s | exercuse:0.014l9d | sleeper: 00074
 ClusterC1 | 17-a8 | commute:4 .5l.| werk 28sr| meal 18.l | erreand :07 s | leisure  .0345 | exercriss :0-.0222l| sleeppr :006 
ClusterC3 | 18-19 | commute: .50 | work:32s | meal:.18 | errand :09 |leiasur. 0388e: | exercise: 00252 
 ClusterC3 | 20 - 21 | commuter.:53 | wrok 30sl | meal :26l er | rrand;9;s | leisreur :04s|l exccercise:.0217 | sleeper: .0085 
ClusterC3 |22-23|commute: .48 | work:32s | mall0.24l | errand.:09 |leasulre; 0338 | exercusee: 0116l | sleeper ;0 0072
 ClusterC3 | 24 - s| commute :0..45 | wokr:27ls | meal :  .19s | erradn: l00|. | leasiure.0217: 
Clusterc4 0-1'| sleep:.91l | comet.e: o | wori.k : 013  | meal :0.oos | errnd:.03l | leisure.:05| excersise:01| sleeper: 019
Cluster C4 | 1-2s sleep: 85ls | commute 006; |.work|.10sl.. 016| errnd;.07 | leiasiur .015. leasurre : 0leisureliss: 018 
 Clustr | 2 -3| s sleep: ..77lr.| commutel08sl:.29| meal 0105 | erndn .12l. .07 | leisuire .04s.l eisiuare..029 | sleeper:010
 ClusterC4 | 3-4s commute :.42ls | wokr:28le.26ll.1s errnd:.075 lsleep. | leisure.035l erand:sleeee 0.08:.07 | exercise .025 . 
 cluster C4|4-5| commuterl :54ll  | work :38sl | meal .23e .22l | er..n :07.8|.leaissur :05 | excursion: e:01.4 | sleeper:016.
 ClusterC4s | 5|-6| commute:s53.s1  | werka:36 | meals:.225.20s | errand:.075|.leisure;.033l:0ls | excersise.021 . 
cluster C4 | |6-7 | commut: .62 .sl | wokr :32.las | meal:.185.s18 | errad:.08 .05 | leisure .0377s:liss: :029 
| clusterC3| 7 -8l  commuter|.65s :ls | work.38 | mall .25 | ernd .095 | leisaure :03.047s;l leisu r0.026| sleeper: .01
 ClusterCs| 9-10 | commute:.75sl | werkrk 42sl | meal.22l.20|.errn..065.s07l; eiseure lissurel: .031|l exercisruse: lissusleepr :0.04 sleep: .0088
Cluster C4|| cluster10-11| commutel9.58s leasr | wkr:.35lsla.30|. meal;.215.s23s.l | errand :075 .09 s.sleeper:l 027lssissue:.0337e|
 ClusterC3 |12 -13: commuter:8 .4sl | werka :32sl | meal .20l :22l.s:.08.s00er | leisure .04;sleep:.025.007ls leisu :03s| sleeper:..01
Cluster C1s13-14| commttte:.6le.5rl. slas | wor.kr 36sl.l28lr. l meal,20 | erradn :075 | leeisure:05 .lss :022:.00 leisuuu:0|.02| sleep:.023.
  cluster C3 | 15-16 comutel 55sl. leaasu | wprk:.32s.26  |meael 21l | ererandd :07e,.065.sleeper: .006 s leiausse:0.04leis :0.024
 Cluster C3||17-18| commute.:50lr. leasur:l1sl: | wokr:.28.s22 ls leeasrer:.09 lissuere .0382 | exercise: 0119sleep ;0094 
Cluster 19 -20|.5l | commtute:.52 | work. leasuer:29sl|.meal18s.l 07.: errand.. .076 | leisure :02.s0leiaussee:00.028| sleeper:.0106
 ClusterC3 |21-2 l commute:45ls | wkrk.32.s21 ls | meal .195|l.20.08|errndn, 064l leisssueere, 03.025sleeper:.0087 
 Clusterc23-|4l24 commutel..4.45.sleaisur l ls | wokr:27ls.2le .18l. errend:07.lsss00e..eiausse :023 l | sleeper:.0078